## DRWFitMetric run on multiple opsims

**Author(s):** Weixiang Yu & Gordon Richards
<br>**Last updated:** 03/10/2021
<br>**Short description:**
This notebook runs the 'DRWFitMetric' on 20 selected cadences and observe the result.

### 0. Software Setup

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import glob
import os
import seaborn as sns
from scipy import stats

In [2]:
# automatically extract username
your_username = os.getcwd().split('/')[5]
print(f'Your automatically extracted username is: {your_username}.'
        '\nIf it is incorrect, please mannually reset it.')

Your automatically extracted username is: ywx649999311.
If it is incorrect, please mannually reset it.


##### Import the sims_maf modules needed.

In [3]:
# import lsst.sim.maf moduels modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
from lsst.sims.maf.stackers import BaseStacker
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

# import convenience functions
from opsimUtils import *

### 1. The MAF Metric

In [4]:
from eztao.carma import DRW_term
from eztao.ts import gpSimByTime, drw_fit
from celerite import GP

In [5]:
from lsst.sims.maf.metrics import BaseMetric
from astropy.stats import mad_std

class DRWFitMetric(BaseMetric):
    """DRW Fitting Accuracy Metric."""
    
    # see eqn #4 & #5 in overview paper
    gamma = {'u': 0.038, 'g':0.039, 'r':0.039, 'i':0.039, 'z':0.039, 'y':0.039}
    sigmaSys = {'u':0.0075, 'g':0.005, 'r':0.005, 'i':0.005, 'z':0.0075, 'y':0.0075}
    
    # DRW parameters distribtuion
    drw_mean = [-1.62852339,  4.43635868]
    drw_cov = [[0.17965110256085912, 0.3059181100985638],
               [0.3059181100985638, 1.9114416154496565]]
    
#     rng = np.random.default_rng()
    
    def __init__(self, mag, band, timesCol='observationStartMJD', m5Col='fiveSigmaDepth', 
                 units='mag', **kwargs):
        
        """Init function for metric class.
        
        Args:
            mag(float): The magnitude of the sources.
            band(str): In which band/filter to run this metric.
        """
        
        # Assign metric parameters to instance object
        self.mag = mag
        self.timesCol = timesCol
        self.m5Col = m5Col
        self.filterCol = 'filter'
        self.metricName = f'DRWFit_{mag}_{band}'
        super(DRWFitMetric, self).__init__(col=[self.timesCol, self.m5Col, self.filterCol], metricName=self.metricName, 
                                           units=units, metricDtype='object', **kwargs)

    def compute_err(self, mags, simData):
        """Compute photometric error given m5 depth and source mags."""
        
        band = np.unique(simData[self.filterCol])[0]
        diffM = mags - simData[self.m5Col]
        varRand = (0.04-self.gamma[band])*np.power(10, 0.4*diffM) + \
                  self.gamma[band]*np.power(10, 0.8*diffM)
        
        return np.sqrt(varRand + self.sigmaSys[band]**2)
        
    def run(self, dataSlice, slicePoint=None):
        """Code executed at each healpix pixel to compute the metric"""
        
        # If the total number of visits < 2, mask as bad pixel
        if dataSlice.size < 2:
            return self.badval

        # generate DRW parameter from pre-defined distribution
        log_amp, log_tau = np.random.multivariate_normal(self.drw_mean, self.drw_cov, 1)[0]
        
        try:
            # create eztao kernel and simulated LC
            kernel = DRW_term(log_amp, log_tau)
            sorted_time = np.sort(dataSlice[self.timesCol]) 
            tIn = sorted_time - sorted_time[0]
            
            # adjust sim factor based on median delta_t & sim lc
            median_delta_t = np.median(tIn[1:] - tIn[:-1])

            if median_delta_t < 0.1:
                sim_factor = 1
            else:
                sim_factor = 100
                
            tOut, yOut, yerrOut = gpSimByTime(kernel, 100, tIn, factor = sim_factor)

            # make final time series
            t = tIn
            y = yOut - yerrOut  + self.mag
            yerr = self.compute_err(y, dataSlice)

            # fit simulated data
            best_drw = drw_fit(t, y, yerr, user_bounds=[[-6, 2], [-5, 11]])
        except:
            return self.badval

        return best_drw

### 2. Run on multiple opsims
#### 2.1 Setup connection ospim database

In [6]:
if your_username == '': # do NOT put your username here, put it in the cell at the top of the notebook.
    raise Exception('Please provide your username!  See the top of the notebook.')

dbDir = '/home/idies/workspace/lsst_cadence/FBS_1.5/'
outDir = f'/home/idies/workspace/Temporary/{your_username}/LSST_MAF/wy/Var/DRW/031021/ResultDBs/'
metricDataPath = f'/home/idies/workspace/Temporary/{your_username}/LSST_MAF/wy/Var/DRW/031021/MetricData/'

if not os.path.exists(os.path.abspath(outDir)):
    os.makedirs(os.path.abspath(outDir))
    
if not os.path.exists(os.path.abspath(metricDataPath)):
    os.makedirs(os.path.abspath(metricDataPath))

In [7]:
dbRuns = show_opsims(dbDir)
dbRuns[0:5] # only show first 5 opsims

['dcr_nham1_ugri_v1.5_10yrs',
 'rolling_mod6_sdf_0.20_v1.5_10yrs',
 'wfd_depth_scale0.95_noddf_v1.5_10yrs',
 'u60_v1.5_10yrs',
 'footprint_stuck_rollingv1.5_10yrs']

#### 2.2 Pick one simulation from each family

In [8]:
dbFamil = [run.split('_')[0] for run in dbRuns]
dfRuns = pd.DataFrame({'run':dbRuns, \
                       'runFamil':dbFamil}).sort_values(by='runFamil').reset_index(drop=True)

# pick one run from each family
dfRunPick = dfRuns.drop_duplicates(subset=['runFamil']).reset_index(drop=True)
dfRunPick

,run,runFamil
0,agnddf_v1.5_10yrs,agnddf
1,alt_dust_v1.5_10yrs,alt
2,baseline_samefilt_v1.5_10yrs,baseline
3,bulges_cadence_bs_v1.5_10yrs,bulges
4,daily_ddf_v1.5_10yrs,daily
5,dcr_nham1_ug_v1.5_10yrs,dcr
6,descddf_v1.5_10yrs,descddf
7,filterdist_indx5_v1.5_10yrs,filterdist
8,footprint_no_gp_northv1.5_10yrs,footprint
9,goodseeing_gri_v1.5_10yrs,goodseeing


#### 2.3 Make Metric Bundles

In [9]:
bundleDict = {}
for mag in [22, 24]:
    
    # declare metric, slicer and sql contraint
    drw_metricG = DRWFitMetric(mag, 'g')
    slicer = slicers.HealpixSlicer(nside=32)
    constraintG = 'filter = "g"'
    constraintG += ' and note not like "DD%"'
    constraintG += ' and proposalId = 1'
    
    # make a bundle
    DRW_mbG = metricBundles.MetricBundle(drw_metricG, slicer, constraintG)
    
    # declare u band metric
    drw_metricU = DRWFitMetric(mag, 'u')
    constraintU = 'filter = "u"'
    constraintU += ' and note not like "DD%"'
    constraintU += ' and proposalId = 1'
    
    # make a bundle
    DRW_mbU = metricBundles.MetricBundle(drw_metricU, slicer, constraintU)

    # put into dict
    bundleDict[drw_metricG.metricName] = DRW_mbG
    bundleDict[drw_metricU.metricName] = DRW_mbU

Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes


#### 2.4 Run MAF on multiple opsims using joblib

In [10]:
# import joblib
from joblib import Parallel, delayed

In [ ]:
# define function to run MAF on one opsim which is easily parallelziable. 
def run_mg(run, bundleDict, dbDir, outDir, metricDataPath):
    """
    Function to run pre-defined MAF metrics on one OpSim. 
    
    Args:
        run (str): The OpSim cadence run name.
        bundleDict (dict): A dictionary of MAF metrics.
        dbDir (str): The path to the OpSim databases.
        outDir (str): The path to the resultdb databases.
        metricDataPath (str): The path to the actual metric data (.npz files). 
    """
    rt = ''
    try:
        for key in bundleDict:
            bundleDict[key].setRunName(run)

        # init connection given run name
        opSimDb, resultDb = connect_dbs(dbDir, outDir, dbRuns=[run])
        # make a group
        metricGroup = metricBundles.MetricBundleGroup(bundleDict, opSimDb[run], 
                                                      metricDataPath, 
                                                      resultDb[run], verbose=False)
        metricGroup.runAll()
    
        # close sql db
        opSimDb[run].close()
        resultDb[run].close()
        
    except Exception as e:
        print(f'{run} failed!')
        print(e)
        print('----------------------')
        rt = run
            
    return rt

# placeholder for joblib returned result
rt = []
rt = Parallel(n_jobs=14)(delayed(run_mg)(run, bundleDict, dbDir, outDir, metricDataPath) 
                         for run in dfRunPick.run.values)

# check failed 
failed_runs = [x for x in rt if len(x) > 0]
print(failed_runs)

#### 2.5 Load back results

In [13]:
# import metric evaluations
bundleDicts = {}
resultDbPath = outDir

resultDbsView = getResultsDbs(resultDbPath)
for runName in resultDbsView:
    bundleDicts[runName] = bundleDictFromDisk(resultDbsView[runName], runName, metricDataPath)

Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slicer using NSIDE=32, approximate resolution 109.935565 arcminutes
Healpix slic

In [14]:
# Check keys
dbRuns = list(resultDbsView.keys())
bd_keys = list(bundleDicts[dbRuns[1]].keys())
print(bd_keys)

[(1, 'DRWFit_22_g'), (2, 'DRWFit_24_g'), (3, 'DRWFit_22_u'), (4, 'DRWFit_24_u')]


#### 2.6 Compute reletive entropy (KL divergence)
- use 2d histogram to turn metric values into discrete probability
- generate another sequence to represent the input probability
- compute KL divergence using scipy.stats.entropy

In [22]:
def compute_kdl(mb, dim=2):
    """Compute KL divergence given a metricbundle."""
    
    # DRW parameters distribtuion
    drw_mean = [-1.62852339,  4.43635868]
    drw_cov = [[0.17965110256085912, 0.3059181100985638],
               [0.3059181100985638, 1.9114416154496565]]
    
    # copy the metric data, combin and reshape
    mbValues = mb.metricValues.copy()
    mask = mbValues.mask
    data = mbValues.data[~mask]
    shape = (data.shape[0], dim)
    data_reshaped = np.concatenate(data).reshape(shape)

    # make a 2d histogram of the metric data
    fit_n, fit_x_edges, fit_y_edges = np.histogram2d(np.log(data_reshaped[:, 0]), 
                                                     np.log(data_reshaped[:, 1]),
                                                     range=[[-5, 1], [-2, 10]],
                                                     bins = 50)
    # rotate fit_n
    fit_n = fit_n.T

    # determine center of histogram bins
    fit_x_centers = (fit_x_edges[:-1] + fit_x_edges[1:])/2
    fit_y_centers = (fit_y_edges[:-1] + fit_y_edges[1:])/2

    # make meshgrid
    fit_xs, fit_ys = np.meshgrid(fit_x_centers, fit_y_centers)
    pos = np.dstack((fit_xs, fit_ys))

    # evaluate input 2d gaussian probability at the center of histogram bins
    input_g2d_rv = stats.multivariate_normal(drw_mean, drw_cov)
    qk = input_g2d_rv.pdf(pos)
    
    # compute & return kdl
    return stats.entropy(fit_n.flatten()/np.sum(fit_n), qk.flatten()/np.sum(qk), base=2)

##### Compute the KL divergence for each opsim and each metric combo (magnitude + band)

In [23]:
# placeholder for all KL divergence values
all_klds = {}

# loop through each metric combo
for key in bd_keys:

    klds = {}
    for run in dbRuns:

        # look for the correct combination of metricID and metricName 
        keys = [*bundleDicts[run].keys()]
        metricKey = [elem for elem in keys if elem[1] == key[1]][0]
        
        # assign KL divergence to dict
        klds[run] = compute_kdl(bundleDicts[run][metricKey])
    
    # store KL divergence for this metric combo to dict
    all_klds[key[1]] = klds

##### __Make into a pandas dataframe__
The metric name follows the format of 'DRWFit_{source magnitude}\_{band}'

In [24]:
kld_df = pd.DataFrame(all_klds)

- ##### Rank opsims by metric value evaluated at umag == 22

In [25]:
kld_df.sort_values('DRWFit_22_u')

,DRWFit_22_g,DRWFit_24_g,DRWFit_22_u,DRWFit_24_u
baseline_samefilt_v1.5_10yrs,0.205755,1.015516,0.340101,2.966426
u60_v1.5_10yrs,0.184243,1.234146,0.370941,2.811723
agnddf_v1.5_10yrs,0.184587,1.504437,0.405985,3.919553
spiders_v1.5_10yrs,0.185792,1.511876,0.408553,3.893161
wfd_depth_scale0.90_v1.5_10yrs,0.198859,1.482049,0.411764,4.199391
rolling_mod2_sdf_0.10_v1.5_10yrs,0.192603,1.410622,0.415297,4.127229
third_obs_pt45v1.5_10yrs,0.202937,1.426384,0.415364,4.084250
daily_ddf_v1.5_10yrs,0.203298,1.485119,0.420847,4.244887
greedy_footprint_v1.5_10yrs,0.199355,1.480232,0.426636,3.811037
twilight_neo_mod2_v1.5_10yrs,0.181255,1.447866,0.432235,4.093353


- ##### Rank opsims by metric value evaluated at gmag == 22

In [26]:
kld_df.sort_values('DRWFit_22_g')

,DRWFit_22_g,DRWFit_24_g,DRWFit_22_u,DRWFit_24_u
twilight_neo_mod2_v1.5_10yrs,0.181255,1.447866,0.432235,4.093353
u60_v1.5_10yrs,0.184243,1.234146,0.370941,2.811723
agnddf_v1.5_10yrs,0.184587,1.504437,0.405985,3.919553
spiders_v1.5_10yrs,0.185792,1.511876,0.408553,3.893161
rolling_mod2_sdf_0.10_v1.5_10yrs,0.192603,1.410622,0.415297,4.127229
dcr_nham1_ug_v1.5_10yrs,0.195305,1.853959,0.499444,4.231523
wfd_depth_scale0.90_v1.5_10yrs,0.198859,1.482049,0.411764,4.199391
greedy_footprint_v1.5_10yrs,0.199355,1.480232,0.426636,3.811037
footprint_no_gp_northv1.5_10yrs,0.200922,1.490549,0.438792,3.977241
third_obs_pt45v1.5_10yrs,0.202937,1.426384,0.415364,4.084250


- ##### Rank opsims by metric value evaluated at umag == 24

In [28]:
kld_df.sort_values('DRWFit_24_u')

,DRWFit_22_g,DRWFit_24_g,DRWFit_22_u,DRWFit_24_u
u60_v1.5_10yrs,0.184243,1.234146,0.370941,2.811723
baseline_samefilt_v1.5_10yrs,0.205755,1.015516,0.340101,2.966426
short_exp_2ns_5expt_v1.5_10yrs,0.211408,1.509473,0.459110,3.768150
greedy_footprint_v1.5_10yrs,0.199355,1.480232,0.426636,3.811037
filterdist_indx5_v1.5_10yrs,0.210265,1.469369,0.461146,3.823838
spiders_v1.5_10yrs,0.185792,1.511876,0.408553,3.893161
roll_mod2_dust_sdf_0.20_v1.5_10yrs,0.212517,1.721646,0.455664,3.902478
agnddf_v1.5_10yrs,0.184587,1.504437,0.405985,3.919553
footprint_no_gp_northv1.5_10yrs,0.200922,1.490549,0.438792,3.977241
bulges_cadence_bs_v1.5_10yrs,0.205648,1.636929,0.451049,3.984316


- ##### Rank opsims by metric value evaluated at gmag == 24

In [29]:
kld_df.sort_values('DRWFit_24_g')

,DRWFit_22_g,DRWFit_24_g,DRWFit_22_u,DRWFit_24_u
baseline_samefilt_v1.5_10yrs,0.205755,1.015516,0.340101,2.966426
u60_v1.5_10yrs,0.184243,1.234146,0.370941,2.811723
descddf_v1.5_10yrs,0.209822,1.409750,0.444252,4.142122
rolling_mod2_sdf_0.10_v1.5_10yrs,0.192603,1.410622,0.415297,4.127229
third_obs_pt45v1.5_10yrs,0.202937,1.426384,0.415364,4.084250
goodseeing_gri_v1.5_10yrs,0.203032,1.443988,0.447482,4.039164
twilight_neo_mod2_v1.5_10yrs,0.181255,1.447866,0.432235,4.093353
filterdist_indx5_v1.5_10yrs,0.210265,1.469369,0.461146,3.823838
greedy_footprint_v1.5_10yrs,0.199355,1.480232,0.426636,3.811037
wfd_depth_scale0.90_v1.5_10yrs,0.198859,1.482049,0.411764,4.199391


##### __Remark:__
A few notes to make:
1. The most of the outliers for the 'SFErroMetric' are also outliers here, but only when evaluated at fainter magnitudes (see the dataframe ranked by 'DRWFit_24_g' and 'DRWFit_24_u').  
2. The difference in the survey parameters doesn't seem having a huge effect on determining DRW parameters at the bright end (see the dataframe ranked by 'DRWFit_22_g' and 'DRWFit_22_u'), also notice the small variance in the metric values (leaving no obvious outliers).
3. The 'baseline_samefilt_v1.5' seems to perform really good here but not so much for the 'SFErrorMetric', this is probably caused by more pairs found in 1 day < $\Delta t$ 10 days (as compared to other simulated cadences), thus recovering more DRWs with a decorrelation timescale in that range. However, at the same time, 'baseline_samefilt_v1.5' gives zero pair in many bins at other timescales making the 'SFErrorMetric' relatively large (bad).